In [1]:
import os
from copy import deepcopy
import pandas as pd
import numpy as np
import math
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score
from sklearn.preprocessing import StandardScaler
from scipy.optimize import linear_sum_assignment
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

from duped_modules.dcn_duped import DCNDuped
from duped_modules.dec_duped import DECDuped, IDECDuped
from all_to_all_autoencoder import AllToAllAutoencoder
from all_to_cat_autoencoder import AllToCatAutoencoder
from cat_to_cat_autoencoder import CatToCatAutoencoder
from load_datasets import load_all_datasets

In [2]:
device = "cpu"
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)
np.random.seed(0)

In [3]:
def cluster_accuracy(labels_true, labels_pred):
    # We need to map the labels to our cluster labels
    # This is a linear assignment problem on a bipartite graph
    k = max(len(np.unique(labels_true)), len(np.unique(labels_pred)))
    cost_matrix = np.zeros((k, k))
    for i in range(labels_true.shape[0]):
        cost_matrix[labels_true[i], labels_pred[i]] += 1
    inverted_cost_matrix = cost_matrix.max() - cost_matrix
    row_ind, col_ind = linear_sum_assignment(inverted_cost_matrix)
    return cost_matrix[row_ind, col_ind].sum() / labels_pred.size

In [4]:
def build_block(layers: list, activation_fn: torch.nn.Module = nn.LeakyReLU, output_fn: torch.nn.Module = nn.LeakyReLU, 
                bias: bool = True, batch_norm: bool = False, dropout: float = None):
    block_list = []
    for i in range(len(layers) - 1):
        block_list.append(torch.nn.Linear(layers[i], layers[i + 1], bias=bias))
        if batch_norm:
            block_list.append(torch.nn.BatchNorm1d(layers[i + 1]))
        if dropout is not None:
            block_list.append(torch.nn.Dropout(dropout))
        if activation_fn is not None:
            if (i != len(layers) - 2):
                block_list.append(activation_fn())
            else:
                if output_fn is not None:
                    block_list.append(output_fn())
    return torch.nn.Sequential(*block_list)

def build_autoencoder(input_dim: int, output_dim: int, layer_per_block: int, activation_fn: torch.nn.Module = nn.LeakyReLU, 
                      output_fn: torch.nn.Module = nn.LeakyReLU, bias: bool = True, batch_norm: bool = False, dropout: float = None):
    hidden_dim = max(1, min(round(input_dim/4), round(output_dim/4)))

    encoder_layer_list = list(range(input_dim, hidden_dim - 1, min(-1, -round((input_dim - hidden_dim) / layer_per_block))))
    encoder_layer_list[-1] = hidden_dim
    encoder = build_block(encoder_layer_list, activation_fn, output_fn, bias, batch_norm, dropout)

    decoder_layer_list = list(range(hidden_dim, output_dim + 1, max(1, round((output_dim - hidden_dim) / layer_per_block))))
    decoder_layer_list[-1] = output_dim
    decoder = build_block(decoder_layer_list, activation_fn, output_fn, bias, batch_norm, dropout)

    return encoder, decoder

In [5]:
datasets = load_all_datasets(max_rows=5000)

accuracies = {d.name: {} for d in datasets}
nmis = {d.name: {} for d in datasets}

for d in datasets:
    print(f"{d.name}: Input dim: {d.input_dim}; Cat dim: {d.cat_dim}; Cont dim: {d.cont_dim}")

Abalone: Input dim: 9; Cat dim: 2; Cont dim: 7
Auction Verification: Input dim: 13; Cat dim: 12; Cont dim: 1
Bank Marketing: Input dim: 22; Cat dim: 17; Cont dim: 5
Breast Cancer: Input dim: 45; Cat dim: 45; Cont dim: 0
Census Income: Input dim: 58; Cat dim: 52; Cont dim: 6
Credit Approval: Input dim: 28; Cat dim: 22; Cont dim: 6
Heart Disease: Input dim: 17; Cat dim: 11; Cont dim: 6
Soybean Disease: Input dim: 55; Cat dim: 55; Cont dim: 0


In [6]:
# All Columns to All Columns Autoencoder
for d in datasets:
    print(f"Calculating for {d.name}...")
    encoder, decoder = build_autoencoder(d.input_dim, d.input_dim, max(1, round(math.log2(d.input_dim))))
    print(encoder, decoder)

    ae = AllToAllAutoencoder(encoder, decoder, d.input_dim, d.cat_dim, d.embedding_sizes, device=device)
    ae.fit(d.dataloader, n_epochs=100, lr=0.001)

    cat = torch.tensor(d.df[d.cat_cols].values, dtype=torch.int).detach().to(device)
    cont = torch.tensor(d.df[d.cont_cols].values, dtype=torch.float).detach().to(device)
    features = ae.encode(cat, cont).detach().cpu().numpy()
    kmeans = KMeans(n_clusters=d.n_targets, init="random", n_init=10, max_iter=300, random_state=0, algorithm="lloyd").fit(features)

    nmis[d.name]["All cols AE"] = normalized_mutual_info_score(d.y, kmeans.labels_)
    accuracies[d.name]["All cols AE"] = cluster_accuracy(d.y, kmeans.labels_)

Calculating for Abalone...
Sequential(
  (0): Linear(in_features=9, out_features=7, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=7, out_features=5, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=5, out_features=2, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=2, out_features=4, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=4, out_features=6, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=6, out_features=9, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
)
Epoch 25/100 - Batch Reconstruction loss: 0.405980
Epoch 50/100 - Batch Reconstruction loss: 0.360183
Epoch 75/100 - Batch Reconstruction loss: 0.382928
Epoch 100/100 - Batch Reconstruction loss: 0.417097
Calculating for Auction Verification...
Sequential(
  (0): Linear(in_features=13, out_features=11, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=11

In [7]:
# Categorial Columns to Categorial Columns Autoencoder
for d in datasets:
    print(f"Calculating for {d.name}...")
    encoder, decoder = build_autoencoder(d.cat_dim, d.cat_dim, max(1, round(math.log2(d.cat_dim))))
    print(encoder, decoder)

    ae = CatToCatAutoencoder(encoder, decoder, d.input_dim, d.cat_dim, d.embedding_sizes, device=device)
    ae.fit(d.dataloader, n_epochs=100, lr=0.001)

    cat = torch.tensor(d.df[d.cat_cols].values, dtype=torch.int).detach().to(device)
    cont = d.df[d.cont_cols].values
    features = np.concatenate((ae.encode(cat, cont).detach().cpu().numpy(), cont), axis=1)
    kmeans = KMeans(n_clusters=d.n_targets, init="random", n_init=10, max_iter=300, random_state=0, algorithm="lloyd").fit(features)

    nmis[d.name]["Cat Cols AE"] = normalized_mutual_info_score(d.y, kmeans.labels_)
    accuracies[d.name]["Cat Cols AE"] = cluster_accuracy(d.y, kmeans.labels_)

Calculating for Abalone...
Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=1, out_features=2, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
)
Epoch 25/100 - Batch Reconstruction loss: 0.753698
Epoch 50/100 - Batch Reconstruction loss: 1.030082
Epoch 75/100 - Batch Reconstruction loss: 7.104859
Epoch 100/100 - Batch Reconstruction loss: 8.671434
Calculating for Auction Verification...
Sequential(
  (0): Linear(in_features=12, out_features=10, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=10, out_features=8, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=8, out_features=6, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=6, out_features=3, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=3, out_features=5, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear

In [8]:
# All Columns to Categorial Columns Autoencoder
for d in datasets:
    print(f"Calculating for {d.name}...")
    encoder, decoder = build_autoencoder(d.input_dim, d.cat_dim, max(1, round(math.log2(d.cat_dim))))
    print(encoder, decoder)

    ae = AllToCatAutoencoder(encoder, decoder, d.input_dim, d.cat_dim, d.embedding_sizes, device=device)
    ae.fit(d.dataloader, n_epochs=100, lr=0.001)

    cat = torch.tensor(d.df[d.cat_cols].values, dtype=torch.int).detach().to(device)
    cont = torch.tensor(d.df[d.cont_cols].values, dtype=torch.float).detach().to(device)
    features = np.concatenate((ae.encode(cat, cont).detach().cpu().numpy(), cont.cpu().numpy()), axis=1)
    kmeans = KMeans(n_clusters=d.n_targets, init="random", n_init=10, max_iter=300, random_state=0, algorithm="lloyd").fit(features)

    nmis[d.name]["All cols to cat cols AE"] = normalized_mutual_info_score(d.y, kmeans.labels_)
    accuracies[d.name]["All cols to cat cols AE"] = cluster_accuracy(d.y, kmeans.labels_)

Calculating for Abalone...
Sequential(
  (0): Linear(in_features=9, out_features=1, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=1, out_features=2, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
)
Epoch 25/100 - Batch Reconstruction loss: 0.620699
Epoch 50/100 - Batch Reconstruction loss: 0.586362
Epoch 75/100 - Batch Reconstruction loss: 3.502085
Epoch 100/100 - Batch Reconstruction loss: 15.952024
Calculating for Auction Verification...
Sequential(
  (0): Linear(in_features=13, out_features=11, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=11, out_features=9, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=9, out_features=7, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=7, out_features=5, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=5, out_features=3, bias=True)
  (9): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linea

In [9]:
pd.DataFrame(nmis.values(), index=nmis.keys())

,All cols AE,Cat Cols AE,All cols to cat cols AE
Abalone,0.131344,0.171743,0.175436
Auction Verification,0.016656,0.179803,0.000022
Bank Marketing,0.033389,0.000095,0.013998
Breast Cancer,0.667875,0.400700,0.642674
Census Income,0.015418,0.010652,0.015944
Credit Approval,0.000092,0.003261,0.036075
Heart Disease,0.110886,0.201481,0.138864
Soybean Disease,0.436927,0.579495,0.436487


In [10]:
pd.DataFrame(accuracies.values(), index=accuracies.keys())

,All cols AE,Cat Cols AE,All cols to cat cols AE
Abalone,0.136462,0.147714,0.158726
Auction Verification,0.622614,0.874694,0.789036
Bank Marketing,0.783400,0.752400,0.520400
Breast Cancer,0.942899,0.849195,0.919473
Census Income,0.538400,0.563400,0.585400
Credit Approval,0.509954,0.542113,0.508423
Heart Disease,0.311037,0.401338,0.321070
Soybean Disease,0.336299,0.549822,0.304270
